# Satellite data : CT (Cloud Type)

The aim of this notebook is to show how to read the Cloud Type (CT) satellite data from a data sample.

You will find one NetCDF file per geographic zone ('NW' for North-West of France and 'SE' for South-East of France) and per year. 

For each pixel, the value represents a category of cloud type (16 classes in total):
* 0:No data
* 1:Cloud-free land
* 2:Cloud-free sea
* 3:Snow over land
* 4:Sea ice
* 5:Very low clouds
* 6:Low clouds
* ...

The spatial resolution is 0.03° and the time step is 15 minutes but some dates can be missing (due to problems in original data, so we can do anything).


For more information, cf [documentation](https://meteofrance.github.io/meteonet/english/data/satellite/).

# Note

<font size="4.5">To use <span style="color:blue">**Cartopy**</span>, a library to plot data with basemaps (see cells below), it is necessary to <span style="color:red">activate the internet connection</span> of that notebook (in edit mode, you can find on the right column, in the *Settings* section, a row entitled *Internet*, put the slider bar on **on**).  </font>

In [ ]:
import xarray as xr
import datetime
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import colors

# Input data files are available in the read-only "../input/" directory
# Any results you write to the current directory are saved as output.

## Parameters definition

Select the year you want to study:

In [ ]:
zone = 'NW'  #on Kaggle, only the North-West zone is available
year = '2016' 

## Loading data using parameters

In [ ]:
fname = "/kaggle/input/meteonet/Satellite_CT/Satellite_CT/CT_%s_%s.nc" % (zone,year) #it is about a data sample : a few days in 2016 in NW zone
data = xr.open_dataset(fname)

## Quick field visualisation

The following command shows the CT data on a given zone at different dates (dates indexes : 0,3,5 and 7).

In [ ]:
# information about color palette
nbClasses = 16
CLOUD_TYPE_16 = {
    "0":"No data",
    "1":"Cloud-free land",
    "2":"Cloud-free sea",
    "3":"Snow over land",
    "4": "Sea ice",
    "5":"Very low clouds",
    "6":"Low clouds",
    "7":"Mid-level clouds",
    "8":"High opaque clouds",
    "9":"Very high opaque clouds",
    "10":"Fractional clouds",
    "11":"High semi transparent thin clouds",
    "12":"High semi transparent meanly thick clouds",
    "13":"High semi transparent thick clouds",
    "14":"High semi transparent above low or medium clouds",
    "15":"High semi transparent above snow/ice"
}

CT_16_COLOR_MAP = [[0.39215686, 0.39215686, 0.39215686],
                   [0., 0.47058824, 0.],
                   [0., 0., 0.],
                   [0.98039216, 0.74509804, 0.98039216],
                   [0.8627451, 0.62745098, 0.8627451],
                   [1., 0.39215686, 0.],
                   [1., 0.70588235, 0.],
                   [0.94117647, 0.94117647, 0.],
                   [0.84313725, 0.84313725, 0.58823529],
                   [0.90196078, 0.90196078, 0.90196078],
                   [0.78431373, 0., 0.78431373],
                   [0., 0.31372549, 0.84313725],
                   [0., 0.70588235, 0.90196078],
                   [0., 0.94117647, 0.94117647],
                   [0.35294118, 0.78431373, 0.62745098],
                   [1, 0.6, 1]]
color = CT_16_COLOR_MAP
legend = CLOUD_TYPE_16
cmap = colors.ListedColormap(color)
norm = colors.BoundaryNorm([i for i in range(nbClasses+1)], cmap.N)

patches = [mpatches.Patch(color=color[i], label=legend[str(i)]) for i in range(nbClasses)]

In [ ]:
data.isel(time=[0,3,5,7])['CT'].plot(x='lon', y='lat',col='time',cmap=cmap, norm=norm)
plt.show()

## Exploration of metadata

Overview of the data:

In [ ]:
data

Get the information about coordinates (latitude and longitude). It is in degrees (north degrees about latitude and east degrees about longitude).

In [ ]:
coord = 'lon'
data[coord]

In [ ]:
data[coord].values[0:10]

Get the information about the observation times:

In [ ]:
dates = data['time']
dates

Get the information about the CT data:

In [ ]:
d = data['CT']
d_vals = d.values  #get the values 
print(d)

The structure of the data (3 dimensions):
* number of observation times
* number of points in latitude 
* number of points in longitude

In [ ]:
d_vals.shape

## Plotting data with Cartopy

Plot the data for one given date with Cartopy:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#studied date
studied_date = '201601010245'
date = datetime.datetime.strptime(studied_date,'%Y%m%d%H%M')

In [ ]:
# Coordinates of studied area boundaries (in °N and °E)
lllat = 46.25  #lower left latitude
urlat = 51.896  #upper right latitude
lllon = -5.842  #lower left longitude
urlon = 2  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

fig = plt.figure(figsize=(8,9))

# Select projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data
img = plt.imshow(data.sel(time=date)['CT'], interpolation='none', origin='upper',cmap=cmap, norm=norm, extent=extent)

# Add coastlines and borders
ax.coastlines(resolution='50m', linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Show only the area we defined
ax.set_extent(extent)

plt.colorbar(img, orientation= 'horizontal', pad=0.03).set_label('Cloud Type (CT)')
plt.legend(handles=patches, loc='lower center',ncol=2,bbox_to_anchor=(0.12, -0.5, 0.8, 0.4),fontsize = 'x-small')
plt.title('CT data - '+ zone + ' zone - '+ studied_date)
plt.show()